In [3]:
import requests

try:
    r = requests.get("https://api.inaturalist.org/v1/observations", timeout=10)
    print("Bağlantı başarılı! Statü kodu:", r.status_code)
except requests.exceptions.RequestException as e:
    print("Bağlantı hatası:", e)


Bağlantı başarılı! Statü kodu: 200


In [8]:
import os
import requests

def get_mushroom_images(query='', per_page=200, max_pages=5, quality_grade='research', min_width=800, min_height=800):
    url = "https://api.inaturalist.org/v1/observations"
    images = []
    
    for page in range(1, max_pages + 1):
        params = {
            "q": query,
            "taxon_name": "Mushrooms",
            "per_page": per_page,
            "page": page,
            "order_by": "desc",
            "order": "observed_on"
        }
        
        response = requests.get(url, params=params)
        
        if response.status_code == 200:
            data = response.json()
            for result in data['results']:
                # Sadece araştırma kalitesindeki gözlemleri al
                if result.get('quality_grade') != quality_grade:
                    continue
                if 'photos' in result:
                    for photo in result['photos']:
                        # Fotoğraf çözünürlüğü en az 800x800 ise
                        width = photo.get('original_dimensions', {}).get('width', 0)
                        height = photo.get('original_dimensions', {}).get('height', 0)
                        if width >= min_width and height >= min_height:
                            images.append(photo['url'])
        else:
            print(f"Error on page {page}: {response.status_code}")
            break
        
        # Eğer yeterli veri toplandıysa, döngüyü kır
        if len(images) >= per_page * max_pages:
            break
    
    return images

def download_images(image_urls, save_folder):
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    for i, url in enumerate(image_urls):
        try:
            response = requests.get(url, timeout=30)  # Timeout ekledim
            if response.status_code == 200:
                image_path = os.path.join(save_folder, f"mantarimage_{i+1}.jpg")
                with open(image_path, 'wb') as file:
                    file.write(response.content)
                print(f"Downloaded {image_path}")
            else:
                print(f"Failed to download image {i+1}")
        except Exception as e:
            print(f"Error downloading image {i+1}: {e}")

def get_and_download_mushrooms():
    # Zehirli ve Zehirsiz mantar türleri
    poisonous_mushrooms = [
    "Amanita bisporigera", "Amanita virosa", "Amanita flavoconia",
    "Amanita muscaria var. flavivolvata", "Amanita pantherina var. pantherinoides",
    "Amanita citrina var. citrina", "Amanita phalloides var. alba",
    "Cortinarius orellanus var. fulvipes", "Lepiota subincarnata var. fastigiata",
    "Cortinarius sanguineus var. sanguineus", "Galerina autumnalis var. spissica",
    "Gymnopilus underwoodii", "Inocybe dulciolens", "Inocybe fuscidula",
    "Omphalotus illudens", "Galerina steglichii", "Hebeloma crustuliniforme var. flavofuscum",
    "Tricholoma terreum", "Panaeolus sphinctrinus", "Hypholoma fasciculare"
]
    
    edible_mushrooms = [
    "Boletus aereus", "Cantharellus cibarius", "Agaricus augustus", "Lactarius torminosus",
    "Clitocybe nebularis", "Morchella esculenta", "Coprinus comatus", "Pleurotus ostreatus",
    "Armillaria mellea", "Laetiporus sulphureus", "Hydnum repandum", "Tricholoma matsutake",
    "Russula vesca", "Fistulina hepatica", "Pholiota adiposa", "Lactarius deliciosus",
    "Hygrophorus russula", "Boletus edulis", "Coprinellus micaceus", "Agaricus bisporus"
]
    
    # Her iki kategori için görselleri al
    poisonous_images = []
    edible_images = []
    
    # Zehirli mantarlar için görsel çekme
    for mushroom in poisonous_mushrooms:
        print(f"Fetching images for poisonous mushroom: {mushroom}")
        images = get_mushroom_images(query=mushroom, per_page=200, max_pages=5)
        poisonous_images.extend(images)
    
    # Zehirsiz mantarlar için görsel çekme
    for mushroom in edible_mushrooms:
        print(f"Fetching images for edible mushroom: {mushroom}")
        images = get_mushroom_images(query=mushroom, per_page=200, max_pages=5)
        edible_images.extend(images)
    
    # Zehirli mantarları indir
    print(f"Downloading {len(poisonous_images)} poisonous mushroom images...")
    download_images(poisonous_images, "poisonous_mushrooms")
    
    # Zehirsiz mantarları indir
    print(f"Downloading {len(edible_images)} edible mushroom images...")
    download_images(edible_images, "edible_mushrooms")

# Ana fonksiyonu çalıştırma
get_and_download_mushrooms()


Fetching images for poisonous mushroom: Amanita bisporigera
Fetching images for poisonous mushroom: Amanita virosa
Fetching images for poisonous mushroom: Amanita flavoconia
Fetching images for poisonous mushroom: Amanita muscaria var. flavivolvata
Fetching images for poisonous mushroom: Amanita pantherina var. pantherinoides
Fetching images for poisonous mushroom: Amanita citrina var. citrina
Fetching images for poisonous mushroom: Amanita phalloides var. alba
Fetching images for poisonous mushroom: Cortinarius orellanus var. fulvipes
Fetching images for poisonous mushroom: Lepiota subincarnata var. fastigiata
Fetching images for poisonous mushroom: Cortinarius sanguineus var. sanguineus
Fetching images for poisonous mushroom: Galerina autumnalis var. spissica
Fetching images for poisonous mushroom: Gymnopilus underwoodii
Fetching images for poisonous mushroom: Inocybe dulciolens
Fetching images for poisonous mushroom: Inocybe fuscidula
Fetching images for poisonous mushroom: Omphalot

KeyboardInterrupt: 

In [5]:
import os
import requests

def get_mushroom_images(query='', per_page=200, max_pages=5, quality_grade='research', min_width=800, min_height=800):
    url = "https://api.inaturalist.org/v1/observations"
    images = []

    for page in range(1, max_pages + 1):
        params = {
            "q": query,
            "taxon_name": "Mushrooms",
            "per_page": per_page,
            "page": page,
            "order_by": "desc",
            "order": "observed_on"
        }

        response = requests.get(url, params=params)

        if response.status_code == 200:
            data = response.json()
            for result in data['results']:
                if result.get('quality_grade') != quality_grade:
                    continue
                if 'photos' in result:
                    for photo in result['photos']:
                        width = photo.get('original_dimensions', {}).get('width', 0)
                        height = photo.get('original_dimensions', {}).get('height', 0)
                        if width >= min_width and height >= min_height:
                            images.append(photo['url'])
        else:
            print(f"Error on page {page}: {response.status_code}")
            break

        if len(images) >= per_page * max_pages:
            break

    return images

def download_images(image_urls, save_folder):
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    for i, url in enumerate(image_urls):
        try:
            response = requests.get(url, timeout=30)
            if response.status_code == 200:
                image_path = os.path.join(save_folder, f"image_{i+1}.jpg")
                with open(image_path, 'wb') as file:
                    file.write(response.content)
                print(f"Downloaded {image_path}")
            else:
                print(f"Failed to download image {i+1}")
        except Exception as e:
            print(f"Error downloading image {i+1}: {e}")

def get_poisonous_mushrooms():
    poisonous_mushrooms = [
    "Amanita jacksonii", "Entoloma sinuatum", "Amanita parcivolvata", "Omphalotus olearius",
    "Psilocybe semilanceata", "Hebeloma crustuliniforme", "Amanita subjunquillea", "Amanita orientigemmata",
    "Tricholoma pardinum", "Inocybe geophylla", "Panaeolus cinctulus", "Amanita longipes",
    "Gymnopilus junonius", "Cortinarius caperatus", "Lepiota helveola", "Inocybe patouillardii",
    "Amanita abrupta", "Galerina marginata", "Clitocybe rivulosa", "Cortinarius gentilis"
]


    all_images = []

    for mushroom in poisonous_mushrooms:
        print(f"Fetching images for: {mushroom}")
        images = get_mushroom_images(query=mushroom, per_page=100, max_pages=2)  # isteğe göre sayıyı azaltabilirsin
        all_images.extend(images)

    print(f"Downloading {len(all_images)} images of poisonous mushrooms...")
    download_images(all_images, "poisonous_mushrooms")

# Ana fonksiyonu çalıştır
get_poisonous_mushrooms()


Fetching images for: Amanita jacksonii
Fetching images for: Entoloma sinuatum
Fetching images for: Amanita parcivolvata
Fetching images for: Omphalotus olearius
Fetching images for: Psilocybe semilanceata
Fetching images for: Hebeloma crustuliniforme
Fetching images for: Amanita subjunquillea
Fetching images for: Amanita orientigemmata
Fetching images for: Tricholoma pardinum
Fetching images for: Inocybe geophylla
Fetching images for: Panaeolus cinctulus
Fetching images for: Amanita longipes
Fetching images for: Gymnopilus junonius
Fetching images for: Cortinarius caperatus
Fetching images for: Lepiota helveola
Fetching images for: Inocybe patouillardii
Fetching images for: Amanita abrupta
Fetching images for: Galerina marginata
Fetching images for: Clitocybe rivulosa
Fetching images for: Cortinarius gentilis
Downloaded poisonous_mushrooms\image_1.jpg
Downloaded poisonous_mushrooms\image_2.jpg
Downloaded poisonous_mushrooms\image_3.jpg
Downloaded poisonous_mushrooms\image_4.jpg
Downloa

In [4]:
import os
import requests

def get_mushroom_images(query='', per_page=200, max_pages=5, quality_grade='research', min_width=800, min_height=800):
    url = "https://api.inaturalist.org/v1/observations"
    images = []
    
    for page in range(1, max_pages + 1):
        params = {
            "q": query,
            "taxon_name": "Mushrooms",
            "per_page": per_page,
            "page": page,
            "order_by": "desc",
            "order": "observed_on"
        }
        
        response = requests.get(url, params=params)
        
        if response.status_code == 200:
            data = response.json()
            for result in data['results']:
                # Sadece araştırma kalitesindeki gözlemleri al
                if result.get('quality_grade') != quality_grade:
                    continue
                if 'photos' in result:
                    for photo in result['photos']:
                        # Fotoğraf çözünürlüğü en az 800x800 ise
                        width = photo.get('original_dimensions', {}).get('width', 0)
                        height = photo.get('original_dimensions', {}).get('height', 0)
                        if width >= min_width and height >= min_height:
                            # Daha büyük çözünürlükte fotoğraf URL'sini al
                            images.append(photo['url'])
        else:
            print(f"Error on page {page}: {response.status_code}")
            break
        
        # Eğer yeterli veri toplandıysa, döngüyü kır
        if len(images) >= per_page * max_pages:
            break
    
    return images

def download_images(image_urls, save_folder):
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    for i, url in enumerate(image_urls):
        try:
            response = requests.get(url, timeout=30)  # Timeout ekledim
            if response.status_code == 200:
                image_path = os.path.join(save_folder, f"mantarr_{i+1}.jpg")
                with open(image_path, 'wb') as file:
                    file.write(response.content)
                print(f"Downloaded {image_path}")
            else:
                print(f"Failed to download image {i+1}")
        except Exception as e:
            print(f"Error downloading image {i+1}: {e}")

def get_and_download_mushrooms():
    # Zehirli ve Zehirsiz mantar türleri
    poisonous_mushrooms = [
        "Amanita phalloides", "Amanita muscaria", "Amanita virosa", "Galerina marginata",
        "Cortinarius orellanus", "Gymnopilus underwoodii", "Inocybe dulciolens", 
        "Omphalotus illudens", "Hebeloma crustuliniforme", "Panaeolus spp."
    ]
    
    edible_mushrooms = [
        "Agaricus bisporus", "Boletus edulis", "Cantharellus cibarius", "Pleurotus ostreatus",
        "Morchella esculenta", "Coprinus comatus", "Laetiporus sulphureus", 
        "Fistulina hepatica", "Hydnum repandum", "Tricholoma matsutake"
    ]
    
    # Her iki kategori için görselleri al
    poisonous_images = []
    edible_images = []
    
    # Zehirli mantarlar için görsel çekme
    for mushroom in poisonous_mushrooms:
        print(f"Fetching images for poisonous mushroom: {mushroom}")
        images = get_mushroom_images(query=mushroom, per_page=200, max_pages=5)
        poisonous_images.extend(images)
    
    # Zehirsiz mantarlar için görsel çekme
    for mushroom in edible_mushrooms:
        print(f"Fetching images for edible mushroom: {mushroom}")
        images = get_mushroom_images(query=mushroom, per_page=200, max_pages=5)
        edible_images.extend(images)
    
    # Zehirli mantarları indir
    print(f"Downloading {len(poisonous_images)} poisonous mushroom images...")
    download_images(poisonous_images, "poisonous_mushrooms")
    
    # Zehirsiz mantarları indir
    print(f"Downloading {len(edible_images)} edible mushroom images...")
    download_images(edible_images, "edible_mushrooms")

# Ana fonksiyonu çalıştırma
get_and_download_mushrooms()


Fetching images for poisonous mushroom: Amanita phalloides


KeyboardInterrupt: 